In [1]:
import pytz
import datetime as dt
import lightgbm as lgb

from helper.future_price_retriever import FuturePriceRetriever
from future_arb.pair_trade_strat import PairTradeStrategy
from matplotlib import pyplot as plt

start_date = "20140601"
end_date = dt.datetime.now(pytz.timezone("Asia/Shanghai")).strftime("%Y%m%d")

# Example usage
trading_pair = ["RB", "HC"]
future_price_retriever = FuturePriceRetriever(start_date=start_date)

# Retrieve spread data
rb_hc_day_spread_df = future_price_retriever.get_spread_data(trading_pair, frequency="1d")

C:\Users\yanzh\AppData\Local\Programs\Python\Python313\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  3 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [ ]:
from helper.spread_data_processor import SpreadDataProcessor


lockback_periods = [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]
hist_vol_windows = [2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

# Process the spread data
data_processor = SpreadDataProcessor()
rb_hc_fibo_z_score = data_processor.compute_moving_statistics(rb_hc_day_spread_df, target_col="RB_HC_spread", window=lockback_periods)
rb_hc_fibo_z_score
rb_hc_fibonacci_spread_df = data_processor.compute_historical_volatility(
    rb_hc_fibo_z_score, price_cols=["RB_prices", "HC_prices"], window=lockback_periods
)
rb_hc_fibonacci_spread_df.columns

Index(['HC_prices', 'RB_prices', 'RB_HC_spread', 'z_score_2d', 'z_score_3d',
       'z_score_5d', 'z_score_8d', 'z_score_13d', 'z_score_21d', 'z_score_34d',
       'z_score_55d', 'z_score_89d', 'z_score_144d', 'z_score_233d',
       'z_score_377d', 'HIST_VOL_2_RB_prices', 'HIST_VOL_2_HC_prices',
       'HIST_VOL_3_RB_prices', 'HIST_VOL_3_HC_prices', 'HIST_VOL_5_RB_prices',
       'HIST_VOL_5_HC_prices', 'HIST_VOL_8_RB_prices', 'HIST_VOL_8_HC_prices',
       'HIST_VOL_13_RB_prices', 'HIST_VOL_13_HC_prices',
       'HIST_VOL_21_RB_prices', 'HIST_VOL_21_HC_prices',
       'HIST_VOL_34_RB_prices', 'HIST_VOL_34_HC_prices',
       'HIST_VOL_55_RB_prices', 'HIST_VOL_55_HC_prices',
       'HIST_VOL_89_RB_prices', 'HIST_VOL_89_HC_prices',
       'HIST_VOL_144_RB_prices', 'HIST_VOL_144_HC_prices',
       'HIST_VOL_233_RB_prices', 'HIST_VOL_233_HC_prices',
       'HIST_VOL_377_RB_prices', 'HIST_VOL_377_HC_prices'],
      dtype='object')

In [4]:
rb_hc_fibonacci_spread_df

,HC_prices,RB_prices,RB_HC_spread,z_score_2d,z_score_3d,z_score_5d,z_score_8d,z_score_13d,z_score_21d,z_score_34d,...,HIST_VOL_5_RB_prices,HIST_VOL_5_HC_prices,HIST_VOL_8_RB_prices,HIST_VOL_8_HC_prices,HIST_VOL_13_RB_prices,HIST_VOL_13_HC_prices,HIST_VOL_21_RB_prices,HIST_VOL_21_HC_prices,HIST_VOL_34_RB_prices,HIST_VOL_34_HC_prices
date,,,,,,,,,,,,,,,,,,,,,
2014-06-03,268.00,521.00,253.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-04,269.50,524.25,254.75,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,0.707107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-05,271.50,526.00,254.50,-0.707107,0.440225,0.440225,0.440225,0.440225,0.440225,0.440225,...,0.032551,0.020476,0.032551,0.020476,0.032551,0.020476,0.032551,0.020476,0.032551,0.020476
2014-06-06,278.00,538.25,260.25,0.707107,1.153846,1.454995,1.454995,1.454995,1.454995,1.454995,...,0.171119,0.160421,0.171119,0.160421,0.171119,0.160421,0.171119,0.160421,0.171119,0.160421
2014-06-09,272.00,534.75,262.75,0.707107,0.847054,1.353646,1.353646,1.353646,1.353646,1.353646,...,0.196778,0.299278,0.196778,0.299278,0.196778,0.299278,0.196778,0.299278,0.196778,0.299278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-11,3413.25,3305.50,-107.75,-0.707107,-1.061588,-1.669508,-1.321065,-0.194534,-0.042285,0.513298,...,0.095423,0.081596,0.095080,0.079704,0.103556,0.089092,0.108102,0.104574,0.100605,0.093390
2025-02-12,3400.50,3281.25,-119.25,-0.707107,-1.054184,-1.550405,-2.078480,-1.712635,-1.401604,-0.540403,...,0.098047,0.077007,0.094478,0.075781,0.104548,0.085189,0.108396,0.101753,0.102142,0.093622
2025-02-13,3397.50,3277.75,-119.75,-0.707107,-0.613786,-1.018105,-1.511087,-1.608813,-1.316076,-0.663242,...,0.100704,0.076480,0.091336,0.074314,0.084473,0.069698,0.107956,0.100884,0.099491,0.092020
